We want to add weather data for each station

In [9]:
import pandas as pd
import xarray as xr
import numpy as np
from read_openmeteo import read_openmeteo

meta = pd.read_csv('metadata5000best.csv')
data = pd.read_csv('stations_complete_2016_2019.csv')


meta = meta.set_index('Unnamed: 0')
meta = meta.drop('Unnamed: 0.1', axis = 1)
meta.index.name = 'stacja'
data = data.set_index('date')
data.index.name = 'date'

data_long = data.reset_index().melt(id_vars = 'date', var_name = 'stacja', value_name = 'pm25')
data_long["date"] = pd.to_datetime(data_long["date"], utc=True, format='mixed')
data_long['date'] =data_long['date'].dt.round('H')

data_long


C:\Users\szymo\AppData\Local\Temp\ipykernel_3236\2947834367.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data_long['date'] =data_long['date'].dt.round('H')


,date,stacja,pm25
0,2016-01-01 00:00:00+00:00,DsLegAlRzecz,59.30
1,2016-01-02 00:00:00+00:00,DsLegAlRzecz,54.70
2,2016-01-03 00:00:00+00:00,DsLegAlRzecz,69.00
3,2016-01-04 00:00:00+00:00,DsLegAlRzecz,80.10
4,2016-01-05 00:00:00+00:00,DsLegAlRzecz,100.50
...,...,...,...
89116,2019-12-27 00:00:00+00:00,ZpSzczec1Maj,3.63
89117,2019-12-28 00:00:00+00:00,ZpSzczec1Maj,NaN
89118,2019-12-29 00:00:00+00:00,ZpSzczec1Maj,NaN
89119,2019-12-30 00:00:00+00:00,ZpSzczec1Maj,3.80


Now add weather data using my implemented function

In [ ]:
import time

meta2 = meta.copy()


all_weather = []
elevations = []


for stacja, lon, lat in zip(meta.index, meta['lon'], meta['lat']):
    print(f"Pobieram pogodę dla {stacja} ({lat}, {lon})...")
    
    df_weather, elevation = read_openmeteo(lon, lat)
    
    elevations.append(elevation)
    
    if df_weather.index.name == "date" or df_weather.index.dtype.kind == "M":
        df_weather = df_weather.reset_index()
    
    df_weather["stacja"] = stacja
    all_weather.append(df_weather)
    
    print("Wait 15 seconds before next request...")
    time.sleep(6)

meta2["elevation"] = elevations

df_weather_all = pd.concat(all_weather, ignore_index=True)

df_weather_all["date"] = pd.to_datetime(df_weather_all["date"], utc=True)

df_weather_all["date"] = df_weather_all["date"].dt.tz_convert('Europe/Warsaw')


df_full = pd.merge(data_long, df_weather_all, on=["date", "stacja"], how="inner")

# Wynik:
print(meta2)      # meta z dodanym elevation
print(df_full)    # pełne dane PM2.5 + pogoda

Pobieram pogodę dla DsLegAlRzecz (51.204503, 16.180513)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla DsOsieczow21 (51.31763, 15.431719)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla DsWalbrzWyso (50.768729, 16.269677)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla DsWrocNaGrob (51.103456, 17.059225)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla DsZgorBohGet (51.150391, 15.008175)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla KpBydFieldor (53.151452, 18.132062)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla KpGrudSienki (53.491831, 18.752503)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla KpToruDziewu (53.028647, 18.666103)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Pobieram pogodę dla KpZielBoryTu (53.662117, 17.934017)...
Czekam 15 sekund, aby nie przekroczyć limitu API...
Po

In [ ]:
from IPython.display import display, HTML

df_full.to_csv('stations_weather_2016_2019.csv')
meta2.to_csv('meta_elevation.csv')